# Libraries

In [1]:
import sys

sys.path.insert(0, "/home/leffff/PycharmProjects/LCT_Hack_Yakutiya_2023/venv/lib/python3.10/site-packages")

In [2]:
from joblib import dump, load
from tqdm.auto import tqdm 
tqdm.pandas()
import re 

import pandas as pd
import numpy as np
from datasets import Dataset

import scipy
import sklearn
from sklearn.neighbors import KDTree, NearestNeighbors

import torch
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer

# Retriever Model

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/leffff/PycharmProjects/LCT_Hack_Yakutiya_2023/venv/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings

In [4]:
class Summarizer:
    def __init__(self, 
                 model: AutoModel,
                 tokenizer: AutoTokenizer,
                 device: str = "cuda"):
        super().__init__()
        self.device = device

        self.model = model
        self.tokenizer = tokenizer
        self.model.eval()
        self.model.to(self.device)

        self.WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

        
    def summarize(self, text):
        input_ids = self.tokenizer(
            [self.WHITESPACE_HANDLER(text)],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512
        )["input_ids"].to(self.device)
        
        output_ids = self.model.generate(
            input_ids=input_ids,
            max_length=256,
            no_repeat_ngram_size=2,
            num_beams=4
        )[0]
        
        summary = self.tokenizer.decode(
            output_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )

        return summary
                

In [5]:
summarizer = Summarizer(
    model,
    tokenizer
) 
summarizer.summarize(
    "от «30 » декабря 2022 г. № Об утверждении Порядка возврата неиспользованных остатков средств, предоставленных муниципальным бюджетным и автономным учреждениям из бюджета МО «Мириинский район» Республики Саха (Якутия)в соответствии с абзацем вторым пункта 1 статьи 78.1. и со статьей 78.2 Бюджетного кодекса Российский Федерации В соответствии с частью 18 статьи 30 Федерального законаот 08.05.2010 № 83-Ф3 «О внесении изменений в отдельные законодательные акты Российской Федерации в связи с совершенствованием правового положения государственных (муниципальных) учреждений», частями 3.17, 3.18 статьи 2 Федерального закона от 03.11.2006 №174-ФЗ «Об автономных учреждениях», постановлениями районной Администрации от 24.11.2020 № 1679 «Об утверждении Порядка определения объема и условий предоставления субсидий на иные цели из бюджета МО «Мирнинский район» Республики Саха (Якутия) муниципальным бюджетным и автономным учреждениям», от 15.02.2017 № 0213 «Об утверждении Положения об осуществлении капитальных вложений в объекты муниципальной собственности МО «Мирнинский район» Республики Саха (Якутия)»: 1. Утвердить Порядок возврата неиспользованных остатков средств, предоставленных муниципальным бюджетным и автономным учреждениям из бюджета МО «Мирнинский район» Республики Саха (Якутия) в соответствии с абзацем вторым пункта | статьи 78.1. и со статьей 78.2 Бюджетного кодекса Российской Федерации согласно приложению к настоящему постановлению. 2. Рекомендовать муниципальным образованиям — поселений 'Мирнинского района руководствоваться настоящим Порядком при разработке муниципальных правовых актов, регламентирующих порядок возврата неиспользованных остатков средств, предоставленных муниципальным бюджетным и автономным учреждениямиз местных бюджетовв соответствии с абзацем вторым пункта | статьи 78.1. и со статьей 78.2 Бюджетного кодекса Российской Федерации. 3. Настоящее Постановление распространяется на правоотношения, возникшиес 01.01.2022 года. 4. Признать утратившим силу постановление районной Администрации от 24.12.2020 № 2059 «Об утверждении Порядка возврата неиспользованных остатков субсидий на иные цели, предоставленных из бюджета МО «Мирнинский район» Республики Саха (Якутия) муниципальным бюджетным и автономным учреждениям». 5. Финансовому управлению (Чемчосва Я.П.) совместно с пресс- службой (Гибало А.О.) разместить настоящее постановление на официальном сайте МО «Мирнинский район» Республики Саха (Якутия) (уу. алмазный край.рф). 6. Контроль исполнения настоящего постановления возложить на заместителя Главы Администрации района по экономике и финансам Башарина Г.К. И.о. Главы Администрации района Д.А. Ширинский Приложение к постановлению районной Администрации от«.4(_» декабря 2022 г.№,03 Порядок возврата неиспользованных остатков средств, предоставленных муниципальным бюджетным и автономным учреждениям из бюджета МО «Мирнинский район»Республики Саха (Якутия) в соответствиис абзацем вторым пункта 1 статьи 78.1 и со статьей 78.2 Бюджетного кодекса Российской Федерации 1. Настоящий Порядок устанавливает правила возврата в бюджет МО «Мирнинский район» Республики Саха (Якутия) неиспользованных на начало текущего финансового года остатков субсидий на иные цели, предоставленных муниципальным бюджетным и автономным учреждениям из бюджета МО «Мирнинский район» Республики Саха (Якутия) в соответствии с абзацем вторым пункта | статьи 78.1. и субсидий на осуществление капитальных вложений в объекты капитального строительства муниципальной собственности и приобретение объектов недвижимого имущества в муниципальную собственность в соответствии со статьей 78.2 Бюджетного кодекса Российской Федерации (далее — субсидии). 2. Возврату подлежат неиспользованные на 1 января текущего финансового года остатки субсидий, предоставленных органами, осуществляющими функции и полномочия учредителя, в течение первых 15 рабочих дней текущего финансового года."
)

'Постановление о возврате неиспользованных остатков средств, предоставленных муниципальным бюджетным и автономным учреждениям из бюджета города "Мирнинский район" Республики Саха (Якутия).'

In [6]:
data = pd.read_csv("parsed.csv")
data

,text,url
0,от«09» (И 2029г. №1 О внесении измененийв пост...,https://www.алмазный-край.рф//upload/files/bnr...
1,от «30 » декабря 2022 г. № Об утверждении Поря...,https://www.алмазный-край.рф//upload/files/bnr...
2,от«. 90 > [2022 т. № О внесении измененийв пос...,https://www.алмазный-край.рф//upload/files/bnr...
3,от«30» 06 утверждении—организационного комитет...,https://www.алмазный-край.рф//upload/files/bnr...
4,от«.30» 12 2022. №2031 О внесении изменений в ...,https://www.алмазный-край.рф//upload/files/bnr...
...,...,...
452,г. Мирный от «(04» 1 р 2025 г. № 1774 О внесен...,https://www.алмазный-край.рф//for%20export/177...
453,от «04»[22023 г. № [177г. Мирный О внесении из...,https://www.алмазный-край.рф//for%20export/177...
454,г. Мирный от «(4» (2 2023 г. № 18 02 О внесени...,https://www.алмазный-край.рф//for%20export/180...
455,г. Мирный от ‹ф 12 2023 г. № 40 О внесении изм...,https://www.алмазный-край.рф//for%20export/180...


In [7]:
data["short_text"] = data["text"].progress_apply(summarizer.summarize)
data

  0%|          | 0/457 [00:00<?, ?it/s]

,text,url,short_text
0,от«09» (И 2029г. №1 О внесении измененийв пост...,https://www.алмазный-край.рф//upload/files/bnr...,В Донецкой области внесены изменения в Порядок...
1,от «30 » декабря 2022 г. № Об утверждении Поря...,https://www.алмазный-край.рф//upload/files/bnr...,Постановление о возврате неиспользованных оста...
2,от«. 90 > [2022 т. № О внесении измененийв пос...,https://www.алмазный-край.рф//upload/files/bnr...,"Администрация района ""Мирнинский район"" Респуб..."
3,от«30» 06 утверждении—организационного комитет...,https://www.алмазный-край.рф//upload/files/bnr...,"Администрация района ""Мирнинский район"" Респуб..."
4,от«.30» 12 2022. №2031 О внесении изменений в ...,https://www.алмазный-край.рф//upload/files/bnr...,Российское министерство образования и науки оп...
...,...,...,...
452,г. Мирный от «(04» 1 р 2025 г. № 1774 О внесен...,https://www.алмазный-край.рф//for%20export/177...,Постановление о создании районной санитарно-пр...
453,от «04»[22023 г. № [177г. Мирный О внесении из...,https://www.алмазный-край.рф//for%20export/177...,Администрация Мирнинского района Республики Са...
454,г. Мирный от «(4» (2 2023 г. № 18 02 О внесени...,https://www.алмазный-край.рф//for%20export/180...,Государственная служба по чрезвычайным ситуаци...
455,г. Мирный от ‹ф 12 2023 г. № 40 О внесении изм...,https://www.алмазный-край.рф//for%20export/180...,Постановление о присвоении звания «Почетный до...


In [11]:
data.to_csv("parsed.csv", index=False)

# Dataset Builder

In [12]:
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

In [25]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [28]:
class RoBERTaEmbeddingBuilder:
    def __init__(self,
                 model: AutoModel,
                 tokenizer: AutoTokenizer,
                 summarizer, 
                 csv_path: str,
                 embeddings_save_path: str = "embeddings.npy",
                 kdtree_save_path: str = "kdtree.joblib",
                 batch_size: int = 1,
                 device: str = "cuda"):
        super().__init__()
        self.device = device

        self.model = model
        self.tokenizer = tokenizer
        self.model.eval()
        self.model.to(self.device)

        self.summarizer = summarizer

        self.embeddings_save_path = embeddings_save_path
        self.kdtree_save_path = kdtree_save_path

        self.batch_size = batch_size

        self._load_dataset(csv_path)
        self._process_dataset()
        self.extract_embeddings()

    def _summarize(self, text):
        return self.summarizer.summarize(text)
    
    def _load_dataset(self, csv_path):
        df = pd.read_csv(csv_path)
        self.dataset = Dataset.from_pandas(df)

    def _process_dataset(self):
        self.dataset = self.dataset.map(
            lambda sample: self._preprocess_text(sample['short_text'])
            # lambda sample: print(sample["text"])
        )

        self.dataset = self.dataset.remove_columns([
            'short_text'
        ])

        self.dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])   
 
    def _preprocess_text(self, text):
        # print(text)
        out = self.tokenizer.encode_plus(text, max_length=512, truncation=True, padding="max_length")
        return out

    def extract_embeddings(self) -> None:
        dataloader = DataLoader(self.dataset, batch_size=self.batch_size)

        embeddings = []

        for batch in tqdm(dataloader):
            input_ids, attention_masks = batch["input_ids"], batch["attention_mask"]
            input_ids, attention_masks = input_ids.to(self.device), attention_masks.to(self.device)

            with torch.no_grad():
                output = mean_pooling(model(
                    input_ids=input_ids,
                    attention_mask=attention_masks,
                ), attention_masks)

            embeddings.append(output.cpu())

        embeddings = torch.cat(embeddings, dim=0).numpy()

        kdtree = NearestNeighbors(n_neighbors=5,
                         metric='cosine',
                         algorithm='brute',
                         n_jobs=-1)
        kdtree.fit(embeddings)
        
        dump(kdtree, self.kdtree_save_path)


In [29]:
builder = RoBERTaEmbeddingBuilder(
    model,
    tokenizer,
    summarizer=summarizer,
    csv_path="parsed.csv",
    embeddings_save_path="embeddings.npy",
    kdtree_save_path="kdtree.joblib",
    batch_size=1,
    device="cuda"
)

Map:   0%|          | 0/457 [00:00<?, ? examples/s]

  0%|          | 0/457 [00:00<?, ?it/s]

# Indexer

In [36]:
class RoBERTaIndexer:
    def __init__(self, 
                 model: AutoModel, 
                 tokenizer: AutoTokenizer, 
                 k: int, 
                 csv_path: str, 
                 kdtree_load_path: str,
                 device: str = "cuda"):
        super().__init__()
        self.device = device

        self.model = model
        self.model.eval()
        self.model.to(self.device)
        self.tokenizer = tokenizer

        self.df = pd.read_csv(csv_path)

        self.k = k

        self.kdtree = self._load_indexer(kdtree_load_path)
        

    def _load_indexer(self, path):
        kdtree = load(path)
        return kdtree
    
    def _preprocess_text(self, text):
        out = self.tokenizer.encode_plus(text, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
        return out

    def _get_embedding(self, sample):
        input_ids = sample["input_ids"]
        if len(input_ids.shape) < 2:
            input_ids.unsqueeze(dim=0)

        attention_mask = sample["attention_mask"]
        if len(attention_mask.shape) < 2:
            input_ids.unsqueeze(dim=0)

        with torch.no_grad():
            return mean_pooling(self.model(input_ids.to(self.device), attention_mask.to(self.device)), attention_mask.to(self.device)).cpu().numpy()

    def search(self, text):
        sample = self._preprocess_text(text)
        embedding = self._get_embedding(sample)

        ind = self.kdtree.kneighbors(embedding, n_neighbors=5, return_distance=False)[0]
        return self.df["short_text"].iloc[ind]

In [37]:
indexer = RoBERTaIndexer(
    model, 
    tokenizer,
    k=5,
    csv_path="parsed.csv",
    kdtree_load_path="kdtree.joblib"
)

In [43]:
indexer.search("стипендии").values

array(['В министерстве образования и науки России внесены изменения в Положение об оплате труда директоров муниципальных учреждений, финансируемых из бюджета.',
       'Президент России Владимир Путин подписал указ о предоставлении единовременной денежной выплаты учителям, прибывшим на работу в муниципальные общеобразовательные организации, и преподавателей, испытывающих проблемы кадрового обеспечения образовательной деятельности.',
       'Мирнинский район Якутии внес изменения в административный регламент при приеме заявлений о зачислении в общеобразовательные организации муниципального образования.',
       'Администрация Мирнинского района Красноярского края приняла в первом чтении Положение о порядке предоставления дополнительных выплат педагогическим работникам филиала ГАПСУ «Якутский медицинский колледж».',
       'Администрация Мирнинского района Москвы приняла в первом чтении изменения в Постановление о расходовании средств целевого финансирования для работников образовательны